In [1]:
import glob, json, argparse, os, sys
import string

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from Bio.PDB import *

sys.path.insert(0,'/data1/groups/keatinglab/swans/savinovCollaboration/inhibitory_fragments_structure_prediction')
from src.colabfold_process_output_utils import *
from src.colabfold_process_output import *
from src.analyze_predictions import *

In [18]:
json_path = '/home/gridsan/sswanson/local_code_mirror/inhibitory_fragments_structure_prediction/colabfold_output.json'

# Load JSON file specifying where to import colabfold results from
# json_path = args.import_json
with open(json_path,"r") as file:
    colab_results = json.loads(file.read())

# For each gene and condition, import available data and create individual dataframes
df_list = []
merge_on_list = ['fragment_name','start','end']
count = 0
for gene_name in colab_results:
    print('gene_name:',gene_name)
    for condition in colab_results[gene_name]:
        print('condition:',condition)
        # Load confidence (pLDDT/iPTM)
        confidence_df = get_confidence_dataframe(colab_results[gene_name][condition]['colabfold'])

        # Count contacts
        contacts_df = get_contact_dataframe(colab_results[gene_name][condition]['colabfold'])

        # TODO calculate RMSD

        # Merge dataframes into a single one containing all types of data
        comb_df = confidence_df.merge(contacts_df,on=merge_on_list,indicator=True,validate="one_to_one")

        # Calculate the weighted contacts
        comb_df['weighted_contacts'] = comb_df['n_contacts'] * comb_df['iptm']

        df_list.append(comb_df)
        count += 1
        if count >= 3:
            break


# Concatenate all dataframes into a single dataframe that stores all the data. Merge with experimental data df 
concat_df = pd.concat(df_list,ignore_index=True)
concat_df.to_csv("colabfold_predictions.csv")
print(f"Dataframe with {len(concat_df)} entries total")

gene_name: folA
condition: 30aa_monomer_folA
gene_name: fstZ
condition: 30aa_monomer_fstZ


KeyboardInterrupt: 

In [12]:
concat_df

,fragment_name,start,aa.fragmentCenter_x,end,iptm,plddt,aa.fragmentCenter_y,n_contacts,path,_merge,weighted_contacts
0,folA_1-159_folA_7-36,7,21.5,36,0.103046,84.774000,21.5,0,/data1/groups/keatinglab/swans/savinovCollabor...,both,0.000000
1,folA_1-159_folA_8-37,8,22.5,37,0.105501,86.700667,22.5,0,/data1/groups/keatinglab/swans/savinovCollabor...,both,0.000000
2,folA_1-159_folA_9-38,9,23.5,38,0.123415,83.461667,23.5,1,/data1/groups/keatinglab/swans/savinovCollabor...,both,0.123415
3,folA_1-159_folA_10-39,10,24.5,39,0.123641,83.581000,24.5,2,/data1/groups/keatinglab/swans/savinovCollabor...,both,0.247281
4,folA_1-159_folA_11-40,11,25.5,40,0.143685,83.008667,25.5,1,/data1/groups/keatinglab/swans/savinovCollabor...,both,0.143685
...,...,...,...,...,...,...,...,...,...,...,...
2391,ssb_1-178_ssb_115-144,115,129.5,144,0.041977,56.419000,129.5,0,/data1/groups/keatinglab/swans/savinovCollabor...,both,0.000000
2392,ssb_1-178_ssb_116-145,116,130.5,145,0.042507,58.255667,130.5,0,/data1/groups/keatinglab/swans/savinovCollabor...,both,0.000000
2393,ssb_1-178_ssb_117-146,117,131.5,146,0.037696,48.113333,131.5,0,/data1/groups/keatinglab/swans/savinovCollabor...,both,0.000000
2394,ssb_1-178_ssb_118-147,118,132.5,147,0.043669,58.134000,132.5,0,/data1/groups/keatinglab/swans/savinovCollabor...,both,0.000000


In [15]:
concat_df[concat_df['_merge']!='both']

,fragment_name,start,aa.fragmentCenter_x,end,iptm,plddt,aa.fragmentCenter_y,n_contacts,path,_merge,weighted_contacts
